In [2]:
import torch
import torch.nn as nn
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from PIL import Image
from tqdm import tqdm as tqdm
import pandas as pd
import os

In [3]:


class CustomResNet50Regression(nn.Module):
    def __init__(self, output_size=1, fine_tune=True):
        super(CustomResNet50Regression, self).__init__()
        # Cargar la ResNet50 preentrenada
        self.resnet50 = models.resnet50(pretrained=True)
        # Reemplazar la última capa para regresión
        self.resnet50.fc = nn.Linear(self.resnet50.fc.in_features, output_size)
        # Congelar capas convolucionales si no se desea fine-tuning
        if not fine_tune:
            for param in self.resnet50.parameters():
                param.requires_grad = False
            # Solo las capas de la fc estarán entrenables
            for param in self.resnet50.fc.parameters():
                param.requires_grad = True

    def forward(self, x):
        return self.resnet50(x)

    def train_model(self, train_loader, val_loader, epochs, learning_rate, fine_tune=True, device='cuda'):
        self.to(device)
        criterion = nn.MSELoss()  # Función de pérdida para regresión
        optimizer = optim.Adam(self.parameters() if fine_tune else self.resnet50.fc.parameters(), lr=learning_rate)

        for epoch in range(epochs):
            # Entrenamiento
            self.train()
            train_loss = 0.0
            for inputs, targets in train_loader:
                inputs, targets = inputs.to(device), targets.to(device)

                optimizer.zero_grad()
                outputs = self(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()

                train_loss += loss.item()

            # Validación
            # self.eval()
            # val_loss = 0.0
            # with torch.no_grad():
            #     for inputs, targets in val_loader:
            #         inputs, targets = inputs.to(device), targets.to(device)
            #         outputs = self(inputs)
            #         loss = criterion(outputs, targets)
            #         val_loss += loss.item()

            print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss/len(train_loader)}, Val Loss: {val_loss/len(val_loader)}")




In [4]:
data_transforms = transforms.Compose([
    transforms.Resize((224)), 
    transforms.Grayscale(num_output_channels=3),  
    transforms.ToTensor(),  # Convertir la imagen a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalización
])
y_train = torch.tensor(pd.read_csv('../data/preprocessed/train/Y_train.csv')['Day30'].values, dtype=torch.float32).unsqueeze(1)
y_test = torch.tensor(pd.read_csv('../data/preprocessed/test/Y_test.csv')['Day30'].values, dtype=torch.float32).unsqueeze(1)

train_images = []
test_images = []

train_img_dir = '../data/preprocessed/train/train_imgs'
train_img_names = os.listdir(train_img_dir)

for img_name in tqdm(train_img_names):
    img_path = os.path.join(train_img_dir, img_name)
    image = Image.open(img_path)
    image = data_transforms(image)  # Aplicar las transformaciones
    train_images.append(image)
train_images_tensor = torch.stack(train_images)

test_img_dir = '../data/preprocessed/test/test_imgs'
test_img_names = os.listdir(test_img_dir)

for img_name in tqdm(test_img_names):
    img_path = os.path.join(test_img_dir, img_name)
    image = Image.open(img_path)
    image = data_transforms(image)  # Aplicar las transformaciones
    test_images.append(image)
test_images_tensor = torch.stack(test_images)

#save tensors y pickle para no tener que hacer esto cada vez

torch.save(train_images_tensor, '../data/preprocessed/train/train_images_tensor.pt')
torch.save(test_images_tensor, '../data/preprocessed/test/test_images_tensor.pt')

  8%|▊         | 1512/18303 [00:13<02:25, 115.29it/s]


KeyboardInterrupt: 

In [ ]:
#load the saved tensors
train_images_tensor = torch.load('../data/preprocessed/train/train_images_tensor.pt')
test_images_tensor = torch.load('../data/preprocessed/test/test_images_tensor.pt')

model = CustomResNet50Regression(output_size=1, fine_tune=True)
train_dataset = torch.utils.data.TensorDataset(train_images_tensor, y_train)
test_dataset = torch.utils.data.TensorDataset(test_images_tensor, y_test)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1000, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1000, shuffle=False)
 
model.train_model(train_loader, test_loader, epochs=10, learning_rate=0.001, fine_tune=False, device='cpu')

/var/folders/t8/cq9vwh6d7dxfyjzswff4ys5m0000gn/T/ipykernel_73018/2800197616.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_images_tensor = torch.load('../data/pre

: 